## Dependency Section

In [ ]:
!pip freeze > requirements.txt
# !pip install -r requirements.txt

In [ ]:
!pip install langsmith chromadb sentence-transformers langchain-community

### Vector DB - Cosine Similarity

## Data Preparation - Adding Metadata

In [ ]:
import json
from uuid import uuid4

def make_chunks(chat_data):
    chunks = []
    i = 0
    while i < len(chat_data):
        if chat_data[i]["speaker"] == "Human":
            human_msg = chat_data[i]["message"]
            ai_msg = chat_data[i + 1]["message"] if (i + 1 < len(chat_data) and chat_data[i + 1]["speaker"] == "AI") else ""
            
            chunk = {
                "id": f"turn_{i}_{uuid4().hex[:6]}",
                "text": human_msg.strip(),
                "metadata": {
                    "speaker": "human",
                    "ai_reply": ai_msg.strip(),
                    "turn_number": i,
                    "emotion": "test",   # placeholder for future tagging, cannot assign null
                    "topic": "test"
                }
            }
            chunks.append(chunk)
        i += 1
    return chunks


# Example
with open("chat.json", "r") as f:
    chat_data = json.load(f)

chunks = make_chunks(chat_data)

with open("chunks_ready.json", "w") as f:
    json.dump(chunks, f, indent=2)

print(f"Created {len(chunks)} chunks ✅")


In [2]:
import json
with open("chunks_ready.json", "r") as f:
    chunks = json.load(f)

print(f"Loaded {len(chunks)} chunks ✅")
print(chunks[0])

Loaded 25 chunks ✅
{'id': 'turn_0_430ba2', 'text': 'I’ve been feeling off lately, like I’m just going through the motions every day.', 'metadata': {'speaker': 'human', 'ai_reply': 'That sounds really hard. When life starts feeling repetitive or dull, it’s often a sign your mind needs a break or some new stimulus. Can you tell me what’s been draining you lately?', 'turn_number': 0, 'emotion': 'test', 'topic': 'test'}}


## Vector Database with L2 and Cosine Similarity

### Vector DB Setup - Euclidean (L2) Similarity

In [ ]:
import os
import json
from uuid import uuid4
import chromadb
from sentence_transformers import SentenceTransformer

# ---------- STEP 3: Initialize persistent Chroma ----------
# Create persistent directory
persist_dir = "./chroma_store"
os.makedirs(persist_dir, exist_ok=True)

# Persistent client
client = chromadb.PersistentClient(path=persist_dir)

# Create or get collection
collection = client.get_or_create_collection("mental_health_chat_mpnet_l2", metadata={"hnsw:space": "l2"})
print("✅ Connected to persistent Chroma collection.")


# ---------- STEP 4: Generate embeddings ----------
# model = SentenceTransformer("all-MiniLM-L6-v2")
model = SentenceTransformer("all-mpnet-base-v2")

ids = [c["id"] for c in chunks]
texts = [c["text"] for c in chunks]
metadatas = [c["metadata"] for c in chunks]

embeddings = model.encode(texts).tolist() 

# ---------- STEP 5: Add to Chroma (if not already present) ----------
existing_ids = set(collection.get(ids=ids).get("ids", []))
new_chunks = [(i, t, m, e) for i, t, m, e in zip(ids, texts, metadatas, embeddings) if i not in existing_ids]

if new_chunks:
    collection.add(
        ids=[i for i, _, _, _ in new_chunks],
        documents=[t for _, t, _, _ in new_chunks],
        metadatas=[m for _, _, m, _ in new_chunks],
        embeddings=[e for _, _, _, e in new_chunks],
    )
    print(f"✅ Added {len(new_chunks)} new chunks to persistent Chroma.")
else:
    print("ℹ️ No new chunks to add. Database already up-to-date.")


# ---------- STEP 6: Example Query ----------
query = "I keep feeling tired and unmotivated lately."
query_emb = model.encode([query]).tolist()

results = collection.query(query_embeddings=query_emb, n_results=1, include=["documents", "metadatas", "distances"])

print("\n🔍 Top Matches:")
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print(f"\n🗣️ Human said: {doc}")
    print(f"🤖 AI replied: {meta['ai_reply']}")
    print("-" * 60)

✅ Connected to persistent Chroma collection.
✅ Added 25 new chunks to persistent Chroma.

🔍 Top Matches:

🗣️ Human said: I’ve been feeling off lately, like I’m just going through the motions every day.
🤖 AI replied: That sounds really hard. When life starts feeling repetitive or dull, it’s often a sign your mind needs a break or some new stimulus. Can you tell me what’s been draining you lately?
------------------------------------------------------------


### Vector DB Setup - Cosine Similarity

In [5]:
import os
import numpy as np
from uuid import uuid4
import chromadb
from sentence_transformers import SentenceTransformer

# ---------- STEP 3: Initialize persistent Chroma ----------
# Create persistent directory
persist_dir = "./chroma_store"
os.makedirs(persist_dir, exist_ok=True)

# Persistent client
client = chromadb.PersistentClient(path=persist_dir)

# Create or get collection
collection = client.get_or_create_collection("mental_health_chat_mpnet_cos", metadata={"hnsw:space": "cosine"})
print("✅ Connected to persistent Chroma collection.")


# ---------- STEP 4: Generate embeddings ----------
model = SentenceTransformer("all-mpnet-base-v2")

ids = [c["id"] for c in chunks]
texts = [c["text"] for c in chunks]
metadatas = [c["metadata"] for c in chunks]

embeddings = model.encode(texts, convert_to_numpy=True)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

embeddings = embeddings.tolist()

# ---------- STEP 5: Add to Chroma (if not already present) ----------
existing_ids = set(collection.get(ids=ids).get("ids", []))
new_chunks = [(i, t, m, e) for i, t, m, e in zip(ids, texts, metadatas, embeddings) if i not in existing_ids]

if new_chunks:
    collection.add(
        ids=[i for i, _, _, _ in new_chunks],
        documents=[t for _, t, _, _ in new_chunks],
        metadatas=[m for _, _, m, _ in new_chunks],
        embeddings=[e for _, _, _, e in new_chunks],
    )
    print(f"✅ Added {len(new_chunks)} new chunks to persistent Chroma.")
else:
    print("ℹ️ No new chunks to add. Database already up-to-date.")


# ---------- STEP 6: Example Query ----------
query = "I keep feeling tired and unmotivated lately."
query_emb = model.encode([query]).tolist()

results = collection.query(query_embeddings=query_emb, n_results=1, include=["documents", "metadatas", "distances"])

print("\n🔍 Top Matches:")
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print(f"\n🗣️ Human said: {doc}")
    print(f"🤖 AI replied: {meta['ai_reply']}")
    print("-" * 60) 

✅ Connected to persistent Chroma collection.
✅ Added 25 new chunks to persistent Chroma.

🔍 Top Matches:

🗣️ Human said: I’ve been feeling off lately, like I’m just going through the motions every day.
🤖 AI replied: That sounds really hard. When life starts feeling repetitive or dull, it’s often a sign your mind needs a break or some new stimulus. Can you tell me what’s been draining you lately?
------------------------------------------------------------


## Retrieval for Vector DB - L2 and Cosine Similarity

### Vector Retrieval - L2 Similarity

In [8]:
import os
import chromadb
from sentence_transformers import SentenceTransformer

# ---------- STEP 1: Connect to Persistent Chroma ----------
persist_dir = "./chroma_store"

# Connect to the existing persistent Chroma store
client = chromadb.PersistentClient(path=persist_dir)

# Load the same collection used before
collection = client.get_or_create_collection("mental_health_chat_mpnet_l2")
print("✅ Connected to persistent Chroma collection.")

# ---------- STEP 2: Load the same embedding model ----------
model = SentenceTransformer("all-mpnet-base-v2")

# ---------- STEP 3: Define a query function ----------
def query_chroma(query_text: str, n_results: int = 3):
    """Query the persistent Chroma DB and print top results."""
    # Generate embedding for query
    query_emb = model.encode([query_text]).tolist()

    # Search in Chroma
    results = collection.query(
        query_embeddings=query_emb,
        n_results=n_results,
        include=["documents", "metadatas", "distances"],
    )

    # Print results
    print(f"\n🔍 Query: {query_text}")
    print("-" * 60)
    for i, (doc, meta, dist) in enumerate(
        zip(results["documents"][0], results["metadatas"][0], results["distances"][0])
    ):
        print(f"🏷️ Match {i+1} (distance: {dist:.4f})")
        print(f"🗣️ Human said: {doc}")
        print(f"🤖 AI replied: {meta.get('ai_reply', 'N/A')}")
        print("-" * 60)

# ---------- STEP 4: Run a sample query ----------
if __name__ == "__main__":
    user_query = "I've been feeling anxious and can't sleep well."
    query_chroma(user_query, n_results=3)


✅ Connected to persistent Chroma collection.

🔍 Query: I've been feeling anxious and can't sleep well.
------------------------------------------------------------


### Vector Retrieval - Cosine Similarity

In [7]:
import os
import chromadb
from sentence_transformers import SentenceTransformer

# ---------- STEP 1: Connect to Persistent Chroma ----------
persist_dir = "./chroma_store"

# Connect to the existing persistent Chroma store
client = chromadb.PersistentClient(path=persist_dir)

# Load the same collection used before
collection = client.get_or_create_collection("mental_health_chat_mpnet_cos")
print("✅ Connected to persistent Chroma collection.")

# ---------- STEP 2: Load the same embedding model ----------
model = SentenceTransformer("all-mpnet-base-v2")

# ---------- STEP 3: Define a query function ----------
def query_chroma(query_text: str, n_results: int = 3):
    """Query the persistent Chroma DB and print top results."""
    # Generate embedding for query
    query_emb = model.encode([query_text]).tolist()

    # Search in Chroma
    results = collection.query(
        query_embeddings=query_emb,
        n_results=n_results,
        include=["documents", "metadatas", "distances"],
    )

    # Print results
    print(f"\n🔍 Query: {query_text}")
    print("-" * 60)
    for i, (doc, meta, dist) in enumerate(
        zip(results["documents"][0], results["metadatas"][0], results["distances"][0])
    ):
        print(f"🏷️ Match {i+1} (distance: {dist:.4f})")
        print(f"🗣️ Human said: {doc}")
        print(f"🤖 AI replied: {meta.get('ai_reply', 'N/A')}")
        print("-" * 60)

# ---------- STEP 4: Run a sample query ----------
if __name__ == "__main__":
    user_query = "I've been feeling anxious and can't sleep well."
    query_chroma(user_query, n_results=3)


✅ Connected to persistent Chroma collection.

🔍 Query: I've been feeling anxious and can't sleep well.
------------------------------------------------------------
🏷️ Match 1 (distance: 0.5093)
🗣️ Human said: I’ve also been sleeping too much, even on weekends.
🤖 AI replied: Oversleeping can be your body’s way of coping with stress or sadness. Would you say your sleep feels restful or more like escape?
------------------------------------------------------------
🏷️ Match 2 (distance: 0.5784)
🗣️ Human said: I’ve been feeling off lately, like I’m just going through the motions every day.
🤖 AI replied: That sounds really hard. When life starts feeling repetitive or dull, it’s often a sign your mind needs a break or some new stimulus. Can you tell me what’s been draining you lately?
------------------------------------------------------------
🏷️ Match 3 (distance: 0.6258)
🗣️ Human said: I’ve been listening to calm piano tracks; they help a bit.
🤖 AI replied: That’s a great start. Music can 

## LangSmith Integration

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key='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')
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)
print(prompt)

In [ ]:
import os
from openai import OpenAI

# The client uses the GROQ_API_KEY environment variable automatically
# You only need to specify the base_url to use Groq's endpoint.
client = OpenAI(
    api_key='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',
    base_url="https://api.groq.com/openai/v1",
)

chat_completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a master of sentiment analysis. Carefully discern the subtle emotions "
                "underlying each interviewer's question. Analyze questions across 8 dimensions: "
                "joy, acceptance, fear, surprise, sadness, disgust, anger, and anticipation. "
                "Score each from 1-10. Your answer must be a valid python list so that it can "
                "be parsed directly, with no extra content! Format: "
                "[{\"analysis\": <REASON>, \"dim\": \"joy\", \"score\": <SCORE>}, ...]"
            )
        },
        {
            "role": "user",
            "content": "I am feeling too lonely, due to which I am unable to sleep at night"
        }
    ],
    temperature=0.2, # Lower temperature is better for structured JSON/List outputs
    max_tokens=1024,
    stream=False,
    stop=None,
)

# Access the result
print(chat_completion.choices[0].message.content)

[{"analysis": "The user is expressing a feeling of loneliness, which is a negative emotion, but the tone is more melancholic than hateful, so the disgust score is relatively low", "dim": "disgust", "score": 2}, {"analysis": "The user is not expressing any joyful emotions, the overall tone is sad and melancholic", "dim": "joy", "score": 1}, {"analysis": "The user is not showing acceptance of their current emotional state, they are expressing a negative feeling", "dim": "acceptance", "score": 3}, {"analysis": "The user is not expressing fear, but rather a state of being that is causing them distress", "dim": "fear", "score": 4}, {"analysis": "There is no surprise in the user's statement, they are simply expressing their emotional state", "dim": "surprise", "score": 1}, {"analysis": "The user is expressing a feeling of loneliness and inability to sleep, which is a sad and melancholic state", "dim": "sadness", "score": 8}, {"analysis": "The user is not expressing anger, but rather a sense 

In [3]:
import json
import ast
from datetime import datetime

# 1. Get the raw content from your Groq completion
raw_output = chat_completion.choices[0].message.content
query_text = "I am feeling too lonely, due to which I am unable to sleep at night"

try:
    # 2. Parse the string into a Python list
    # We use ast.literal_eval as a safer alternative to eval() for python lists/dicts
    # Or json.loads() if the LLM output is strictly valid JSON format
    parsed_data = ast.literal_eval(raw_output.strip())

    # 3. Create the final structured object
    structured_log = {
        "timestamp": datetime.now().isoformat(),
        "query": query_text,
        "model": 'llama-3.3-70b-versatile',
        "emotion_results": parsed_data
    }

    # 4. Output or save the results
    print(json.dumps(structured_log, indent=4))

except Exception as e:
    print(f"Failed to parse LLM output: {e}")
    print(f"Raw output was: {raw_output}")

{
    "timestamp": "2025-12-31T14:11:52.749778",
    "query": "I am feeling too lonely, due to which I am unable to sleep at night",
    "model": "llama-3.3-70b-versatile",
    "emotion_results": [
        {
            "analysis": "The user is expressing a feeling of loneliness, which is a negative emotion, but the tone is more melancholic than hateful, so the disgust score is relatively low",
            "dim": "disgust",
            "score": 2
        },
        {
            "analysis": "The user is not expressing any joyful emotions, the overall tone is sad and melancholic",
            "dim": "joy",
            "score": 1
        },
        {
            "analysis": "The user is not showing acceptance of their current emotional state, they are expressing a negative feeling",
            "dim": "acceptance",
            "score": 3
        },
        {
            "analysis": "The user is not expressing fear, but rather a state of being that is causing them distress",
            "

In [ ]:
# Extract dimension scores
dimension_scores = {
    item["dim"]: item["score"]
    for item in structured_log["emotion_results"]
}
print(dimension_scores)

# Sorted Alphabetically
# acceptance, anger, anticipation, disgust, fear, joy, sadness, surprise

{'disgust': 2, 'joy': 1, 'acceptance': 3, 'fear': 4, 'surprise': 1, 'sadness': 8, 'anger': 2, 'anticipation': 2}


In [10]:
# Define the desired alphabetical order
dimensions = sorted(item["dim"] for item in structured_log["emotion_results"])
print(dimensions)

# Create a mapping of dimension to score
score_map = {item["dim"]: item["score"] for item in structured_log["emotion_results"]}
print(score_map)
# Generate the score list in alphabetical order
score_list = [score_map[dim] for dim in dimensions]

print(score_list)

['acceptance', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
{'disgust': 2, 'joy': 1, 'acceptance': 3, 'fear': 4, 'surprise': 1, 'sadness': 8, 'anger': 2, 'anticipation': 2}
[3, 2, 2, 2, 4, 1, 8, 1]


## Custom Chroma DB For Emotion Embeddings

In [ ]:
import chromadb
from chromadb.config import Settings

# Persistent DB (saved to disk)
client = chromadb.Client(
    Settings(
        persist_directory="./emotion_vector_db",
        anonymized_telemetry=False
    )
)

# Create / get collection
collection = client.get_or_create_collection(
    name="emotion_vectors",
    metadata={"description": "8D emotion embeddings"}
)


In [ ]:
# Example emotion vectors
embeddings = [
    [3, 2, 2, 2, 4, 1, 8, 1],
    [7, 1, 1, 1, 1, 5, 2, 1],
    [1, 6, 3, 4, 2, 1, 5, 2]
]

# Optional: normalize (recommended)
embeddings = [[v / 10 for v in vec] for vec in embeddings]

documents = [
    "User feeling lonely and sleepless",
    "User feeling calm and content",
    "User feeling angry and anxious"
]

metadatas = [
    {"label": "lonely"},
    {"label": "calm"},
    {"label": "angry"}
]

ids = ["doc1", "doc2", "doc3"]

collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids
)


In [ ]:
client.persist()


In [ ]:
query_vector = [2, 2, 2, 2, 3, 1, 7, 1]
query_vector = [v / 10 for v in query_vector]

results = collection.query(
    query_embeddings=[query_vector],
    n_results=2
)

print(results)


In [ ]:
{
  'ids': [['doc1', 'doc3']],
  'documents': [['User feeling lonely...', 'User feeling angry...']],
  'metadatas': [[{'label': 'lonely'}, {'label': 'angry'}]],
  'distances': [[0.12, 0.34]]
}


In [ ]:
results = collection.query(
    query_embeddings=[query_vector],
    n_results=3,
    where={"label": "lonely"}
)


In [ ]:
collection.update(
    ids=["doc1"],
    embeddings=[[0.3,0.2,0.2,0.2,0.4,0.1,0.8,0.1]],
    documents=["Updated emotional state"]
)


In [ ]:
collection.delete(ids=["doc3"])


In [ ]:
vec = [v / 10 for v in vec]


In [ ]:
assert len(vec) == 8